# adapted mismap function to run on unique list of ENSP sequences mapping to same UniProt ID

**see Pmap_mismap_funx.py for both versions of function**

11.25.19

mfpfox

In [10]:
# packages
import os
import sys
import numpy as np
import pandas as pd
from statistics import stdev 
from ast import literal_eval # for mismap_score func
from statistics import mean

In [11]:
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000
display(HTML("<style>.container {width:90% !important;}</style>"))
sys.path.append("/Users/mariapalafox/Desktop/Toolbox")
from all_funx import *

In [12]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MULTIMAPPING_IDS/MISMAP2.0/MISMAP_UNIQUE_ENSP_EDITION/")

In [13]:
# from Pmap_make_groupby_df_sets.py
def setme(s):
    ls = list(s)
    finalset = set(ls)
    return finalset

def groupby_df(df):
    # columns based on set of IDs or sequences from all releases
    ENSP_df = df.groupby('xref')['ENSP'].apply(setme)
    ENSPv_df = df.groupby('xref')['ENSPv'].apply(setme)
    ENST_df = df.groupby('xref')['ENST'].apply(setme)
    ENSTv_df = df.groupby('xref')['ENSTv'].apply(setme)
    ENSG_df = df.groupby('xref')['ENSG'].apply(setme)
    ENSGv_df = df.groupby('xref')['ENSGv'].apply(setme)
    stableID_key_df = df.groupby('xref')['stableID_key'].apply(setme)
    proSequence_df = df.groupby('xref')['proSequence'].apply(setme)
    # concat
    set_columns = pd.concat([ENSP_df, ENSPv_df, ENST_df, ENSTv_df, ENSG_df, ENSGv_df, stableID_key_df, proSequence_df], axis=1)
    # counting length of sets aka # unique values in each column
    set_columns['count_ENSP'] = set_columns['ENSP'].apply(lambda x: len(x))
    set_columns['count_ENSPv'] = set_columns['ENSPv'].apply(lambda x: len(x))
    set_columns['count_ENST'] = set_columns['ENST'].apply(lambda x: len(x))
    set_columns['count_ENSTv'] = set_columns['ENSTv'].apply(lambda x: len(x))
    set_columns['count_ENSG'] = set_columns['ENSG'].apply(lambda x: len(x))
    set_columns['count_ENSGv'] = set_columns['ENSGv'].apply(lambda x: len(x))
    set_columns['count_stableID_key'] = set_columns['stableID_key'].apply(lambda x: len(x))
    set_columns['count_proSequence'] = set_columns['proSequence'].apply(lambda x: len(x))
    # columns based on list of things NOT SET, for SD calculation
    Length_df = df.groupby('xref')['Length'].apply(list)
    prov_df = df.groupby('xref')['pro_ver'].apply(list)
    txv_df = df.groupby('xref')['tx_ver'].apply(list)
    genv_df = df.groupby('xref')['gen_ver'].apply(list)
    # concat
    list_columns = pd.concat([Length_df, prov_df, txv_df, genv_df], axis=1)
    # creating column for number of ENSP ids
#     list_columns['stdev_length'] = list_columns['Length'].apply(lambda x: stdev(x))
#     list_columns['stdev_prov'] = list_columns['pro_ver'].apply(lambda x: stdev(x))
#     list_columns['stdev_txv'] = list_columns['tx_ver'].apply(lambda x: stdev(x))
#     list_columns['stdev_genv'] = list_columns['gen_ver'].apply(lambda x: stdev(x))
    # merging set df and list df 
    final = pd.concat([set_columns, list_columns], axis=1)
    print("final df shape: ", final.shape)
    print("preview: ")
    print(final.head(1))
    return final

In [14]:
# main function
v85 = pd.read_csv("v85_fasta_merge_xref_3953IDs_10183.csv")
v92 = pd.read_csv("v92_fasta_merge_xref_3953IDs_10395.csv")
v94 = pd.read_csv("v94_fasta_merge_xref_3953IDs_10612.csv")
v96 = pd.read_csv("v96_fasta_merge_xref_3953IDs_10663.csv")
v97 = pd.read_csv("v97_fasta_merge_xref_3953IDs_10564.csv")

In [15]:
# main funx
v85final = groupby_df(v85)
v92final = groupby_df(v92)
v94final = groupby_df(v94)
v96final = groupby_df(v96)
v97final = groupby_df(v97)

final df shape:  (3953, 20)
preview: 
                                      ENSP  \
xref                                         
A0AVT1  {ENSP00000313454, ENSP00000399234}   

                                         ENSPv  \
xref                                             
A0AVT1  {ENSP00000399234.2, ENSP00000313454.4}   

                                      ENST  \
xref                                         
A0AVT1  {ENST00000420827, ENST00000322244}   

                                         ENSTv               ENSG  \
xref                                                                
A0AVT1  {ENST00000420827.2, ENST00000322244.5}  {ENSG00000033178}   

                      ENSGv  \
xref                          
A0AVT1  {ENSG00000033178.8}   

                                             stableID_key  \
xref                                                        
A0AVT1  {ENSG00000033178_ENST00000322244_ENSP000003134...   

                                              p

## errors found in excel view from IDs: Q8WZ42 (19 pro sequence all 2k) & Q15149 (18 proseeq all 4k)
**solution=save mismap scored files without the protein sequence**

In [9]:
v85final.to_csv("problem_df85_check.csv")

## convert column of sets into column of lists for mismap function & reset index

In [16]:
dfs = [v85final, v92final, v94final, v96final, v97final]
for i in dfs: 
    i['proSequence'] = i['proSequence'].apply(list)
    i.reset_index(inplace=True)

In [17]:
# change xref col name to ID
dfs = [v85final, v92final, v94final, v96final, v97final]
for i in dfs: 
    i.columns = ['ID', 'ENSP', 'ENSPv', 'ENST', 'ENSTv', 'ENSG', 'ENSGv',
       'stableID_key', 'proSequence', 'count_ENSP', 'count_ENSPv',
       'count_ENST', 'count_ENSTv', 'count_ENSG', 'count_ENSGv',
       'count_stableID_key', 'count_proSequence', 'Length', 'pro_ver',
       'tx_ver', 'gen_ver']


In [18]:
dfs = [v85final, v92final, v94final, v96final, v97final]
for i in dfs: 
    print(i.shape)

(3953, 21)
(3953, 21)
(3953, 21)
(3953, 21)
(3953, 21)


# import ultimate ukb key - adapting mismap score to work on columns with unique sequence lists

In [19]:
reference = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MULTIMAPPING_IDS/ultimate_UKB_sequence_key_3953.csv")

In [20]:
# droppign the columns i dont need, like proSequence and in3979xref
UKB = reference[['ID', 'entryName', 'labeled_pos_count', 'pos_dict', 'count_C_targets', 'count_K_targets']].copy()
UKB.head(3)

,ID,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets
0,Q9HAS0,NJMU_HUMAN,1,{182: 'C'},1,0
1,Q86X76,NIT1_HUMAN,3,"{161: 'K', 165: 'C', 203: 'C'}",2,1
2,Q9NQR4,NIT2_HUMAN,7,"{44: 'C', 52: 'K', 123: 'K', 130: 'K', 146: 'C...",4,3


In [21]:
# adapted from F and 
def list2col(s, colname):
    fseries = pd.DataFrame(np.array(s).reshape(-1,1))
    fseries.columns = [colname]
    return fseries

def mismap_score(enspdf, uniprot):
    # new columns to add
    count_col = []
    count_cys_col = []
    count_lys_col = []

    miss_col = []
    miss_count_c = []
    miss_count_k = []

    corrFrac_col = []
    missFrac_col = []

    # merge two df
    mer = pd.merge(enspdf, uniprot, how='inner', on=['ID'])

    # QC merge
    print('shape ENSEMBL df: ', enspdf.shape)
    print('shape ukb df: ', uniprot.shape)
    print('shape post merge: ', mer.shape)

    for index, row in mer.iterrows():
        # pep is a list of seqeunces
        pep = row['proSequence']
        total_proseq = int(row['count_proSequence']) # number of unique ensp prosequences
        ukb_id = row['ID']
        posCK = row['pos_dict']

        # multiplying # per ukb ID by number of ENSP sequences mapping to UKB ID
        tot_tar = int(row['labeled_pos_count']) * total_proseq
        tot_c = int(row['count_C_targets']) * total_proseq
        tot_k = int(row['count_K_targets']) * total_proseq

        # positions found counter, reset with new row/UKBID
        count = 0
        count_cys = 0
        count_lys = 0
        
        # evaluate as dictionary
        python_dict = literal_eval(posCK)

        # looping thru prosequence list
        for proseq in pep:
            ensp_len = len(proseq)
            
            # loop thru position dict
            for key in python_dict:
                k = int(key)
                # convert pos # to index 0-based
                i = k - 1
                # if index within ensembl protein string
                if i < int(ensp_len):
                    # get AA from peptide string
                    AA = proseq[i]
                    # get AA from pos dic
                    checker = python_dict[key]
                    # match, increase count
                    if AA == checker:
                        count += 1
                        if AA == 'C':
                            count_cys += 1
                        if AA == 'K':
                            count_lys += 1

        # fraction correct pos
        cor_per = count/tot_tar  # frac corr is total found / # pos in ukb IDs * # ensp seq checked

        # fraction missed
        miss = tot_tar - count
        missC = tot_c - count_cys
        missK = tot_k - count_lys

        miss_per = miss/tot_tar

        # add columns
        count_col.append(count)
        count_cys_col.append(count_cys)
        count_lys_col.append(count_lys)

        miss_col.append(miss)
        miss_count_c.append(missC)
        miss_count_k.append(missK)

        corrFrac_col.append(round(cor_per, 3))
        missFrac_col.append(round(miss_per, 3))


    # add columns
    count_col2 = list2col(count_col, "found_count")
    count_cys_col2 = list2col(count_cys_col, "found_count_C")
    count_lys_col2 = list2col(count_lys_col, "found_count_K")

    miss_col2 = list2col(miss_col, "missed_count")
    miss_count_c2 = list2col(miss_count_c, "missed_count_C")
    miss_count_k2 = list2col(miss_count_k, "missed_count_K")

    corrFrac_col2 = list2col(corrFrac_col, "correct_frac")
    missFrac_col2 = list2col(missFrac_col, "missed_frac")

    df = pd.concat([mer, count_col2,count_cys_col2, count_lys_col2, miss_col2,miss_count_c2, miss_count_k2, corrFrac_col2, missFrac_col2], axis=1)
    print("final df shape: ", df.shape)
    print()
    return df

In [22]:
df85 = mismap_score(v85final, UKB)
df92 = mismap_score(v92final, UKB)
df94 = mismap_score(v94final, UKB)
df96 = mismap_score(v96final, UKB)
df97 = mismap_score(v97final, UKB)

shape ENSEMBL df:  (3953, 21)
shape ukb df:  (3953, 6)
shape post merge:  (3953, 26)
final df shape:  (3953, 34)

shape ENSEMBL df:  (3953, 21)
shape ukb df:  (3953, 6)
shape post merge:  (3953, 26)
final df shape:  (3953, 34)

shape ENSEMBL df:  (3953, 21)
shape ukb df:  (3953, 6)
shape post merge:  (3953, 26)
final df shape:  (3953, 34)

shape ENSEMBL df:  (3953, 21)
shape ukb df:  (3953, 6)
shape post merge:  (3953, 26)
final df shape:  (3953, 34)

shape ENSEMBL df:  (3953, 21)
shape ukb df:  (3953, 6)
shape post merge:  (3953, 26)
final df shape:  (3953, 34)



In [23]:
df85

,ID,ENSP,ENSPv,ENST,ENSTv,ENSG,ENSGv,stableID_key,proSequence,count_ENSP,count_ENSPv,count_ENST,count_ENSTv,count_ENSG,count_ENSGv,count_stableID_key,count_proSequence,Length,pro_ver,tx_ver,gen_ver,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac
0,A0AVT1,"{ENSP00000313454, ENSP00000399234}","{ENSP00000399234.2, ENSP00000313454.4}","{ENST00000420827, ENST00000322244}","{ENST00000420827.2, ENST00000322244.5}",{ENSG00000033178},{ENSG00000033178.8},{ENSG00000033178_ENST00000322244_ENSP000003134...,[MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYS...,2,2,2,2,1,1,2,2,"[1052, 389]","[4, 2]","[5, 2]","[8, 8]",UBA6_HUMAN,11,"{86: 'K', 347: 'C', 409: 'K', 433: 'C', 544: '...",6,5,13,7,6,9,5,4,0.591,0.409
1,A0FGR8,{ENSP00000251527},{ENSP00000251527.5},{ENST00000251527},{ENST00000251527.5},{ENSG00000117868},{ENSG00000117868.11},{ENSG00000117868_ENST00000251527_ENSP00000251527},[MTPPSRAEAGVRRSRVPSEGRWRGAEPPGISASTQPASAGRAARH...,1,1,1,1,1,1,1,1,[893],[5],[5],[11],ESYT2_HUMAN,1,{181: 'C'},1,0,0,0,0,1,1,0,0.000,1.000
2,A0JNW5,"{ENSP00000349285, ENSP00000444824, ENSP0000027...","{ENSP00000349285.3, ENSP00000444824.2, ENSP000...","{ENST00000545232, ENST00000356828, ENST0000027...","{ENST00000545232.2, ENST00000279907.7, ENST000...",{ENSG00000111647},{ENSG00000111647.8},{ENSG00000111647_ENST00000545232_ENSP000004448...,[MQLSFTQLTIDYYPYHKAGDSCNHWMYFSDATKTKNGWANELLHE...,3,3,3,3,1,1,3,3,"[1464, 522, 1114]","[7, 3, 2]","[7, 3, 2]","[8, 8, 8]",UH1BL_HUMAN,1,{645: 'K'},0,1,1,0,1,2,0,2,0.333,0.667
3,A0MZ66,"{ENSP00000376636, ENSP00000260777, ENSP0000037...","{ENSP00000347532.4, ENSP00000376636.2, ENSP000...","{ENST00000392901, ENST00000355371, ENST0000026...","{ENST00000392903.2, ENST00000392901.4, ENST000...",{ENSG00000187164},{ENSG00000187164.13},{ENSG00000187164_ENST00000355371_ENSP000003475...,[MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQER...,4,4,4,4,1,1,4,4,"[456, 631, 498, 649]","[10, 4, 4, 2]","[10, 4, 4, 2]","[13, 13, 13, 13]",SHOT1_HUMAN,1,{565: 'C'},1,0,2,2,0,2,2,0,0.500,0.500
4,A1KXE4,"{ENSP00000374565, ENSP00000387051}","{ENSP00000387051.1, ENSP00000374565.3}","{ENST00000409185, ENST00000389915}","{ENST00000389915.3, ENST00000409185.1}",{ENSG00000152102},{ENSG00000152102.13},{ENSG00000152102_ENST00000409185_ENSP000003870...,[MNPVYSPGSSGVPYANAKGIGYPAGFPMGYAAAAPAYSPNMYPGA...,2,2,2,2,1,1,2,1,"[195, 195]","[3, 1]","[3, 1]","[13, 13]",F168B_HUMAN,1,{63: 'C'},1,0,1,1,0,0,0,0,1.000,0.000
5,A1L0T0,{ENSP00000263383},{ENSP00000263383.3},{ENST00000263383},{ENST00000263383.3},{ENSG00000105135},{ENSG00000105135.11},{ENSG00000105135_ENST00000263383_ENSP00000263383},[METPAAAAPAGSLFPSFLLLACGTLVAALLGAAHRLGLFYQLLHK...,1,1,1,1,1,1,1,1,[632],[3],[3],[11],ILVBL_HUMAN,1,{538: 'K'},0,1,1,0,1,0,0,0,1.000,0.000
6,A2A288,"{ENSP00000386616, ENSP00000440813, ENSP0000037...","{ENSP00000386616.3, ENSP00000440813.1, ENSP000...","{ENST00000409806, ENST00000542614, ENST0000038...","{ENST00000409806.3, ENST00000389942.5, ENST000...",{ENSG00000178199},{ENSG00000178199.9},{ENSG00000178199_ENST00000542614_ENSP000004408...,[MEHPSKMEFFQKLGYDREDVLRVLGKLGEGALVNDVLQELIRTGS...,3,3,3,3,1,1,3,2,"[527, 527, 321]","[5, 3, 1]","[5, 3, 1]","[9, 9, 9]",ZC12D_HUMAN,3,"{64: 'C', 367: 'C', 517: 'C'}",3,0,4,4,0,2,2,0,0.667,0.333
7,A2RUC4,{ENSP00000346627},{ENSP00000346627.4},{ENST00000354611},{ENST00000354611.4},{ENSG00000162971},{ENSG00000162971.6},{ENSG00000162971_ENST00000354611_ENSP00000346627},[MAGQHLPVPRLEGVSREQFMQHLYPQRKPLVLEGIDLGPCTSKWT...,1,1,1,1,1,1,1,1,[315],[4],[4],[6],TYW5_HUMAN,1,{194: 'K'},0,1,1,0,1,0,0,0,1.000,0.000
8,A3KN83,"{ENSP00000473480, ENSP00000473665, ENSP0000026...","{ENSP00000387361.2, ENSP00000267176.4, ENSP000...","{ENST00000602750, ENST00000267176, ENST0000042...","{ENST00000420886.2, ENST00000267176.4, ENST000...",{ENSG00000139697},{ENSG00000139697.7},{ENSG00000139697_ENST00000602750_ENSP000004734...,[MVEPGQDLLLAALSESGISPNDLFDIDGGDAGLATPMPT

In [24]:
df85.to_csv("df85_unique_prosequence_MISMAP_scored_3953.csv", index=False)
df92.to_csv("df92_unique_prosequence_MISMAP_scored_3953.csv", index=False)
df94.to_csv("df94_unique_prosequence_MISMAP_scored_3953.csv", index=False)
df96.to_csv("df96_unique_prosequence_MISMAP_scored_3953.csv", index=False)
df97.to_csv("df97_unique_prosequence_MISMAP_scored_3953.csv", index=False)

In [26]:
dfs = [df85, df92, df94, df96, df97]
for i in dfs: 
    i.drop('proSequence', axis=1, inplace=True)  
    print(i.columns)
# dropping problematic prosequence column, sets with 18+ proteins all 4k causing excel weird behavior

Index(['ID', 'ENSP', 'ENSPv', 'ENST', 'ENSTv', 'ENSG', 'ENSGv', 'stableID_key',
       'count_ENSP', 'count_ENSPv', 'count_ENST', 'count_ENSTv', 'count_ENSG',
       'count_ENSGv', 'count_stableID_key', 'count_proSequence', 'Length',
       'pro_ver', 'tx_ver', 'gen_ver', 'entryName', 'labeled_pos_count',
       'pos_dict', 'count_C_targets', 'count_K_targets', 'found_count',
       'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C',
       'missed_count_K', 'correct_frac', 'missed_frac'],
      dtype='object')
Index(['ID', 'ENSP', 'ENSPv', 'ENST', 'ENSTv', 'ENSG', 'ENSGv', 'stableID_key',
       'count_ENSP', 'count_ENSPv', 'count_ENST', 'count_ENSTv', 'count_ENSG',
       'count_ENSGv', 'count_stableID_key', 'count_proSequence', 'Length',
       'pro_ver', 'tx_ver', 'gen_ver', 'entryName', 'labeled_pos_count',
       'pos_dict', 'count_C_targets', 'count_K_targets', 'found_count',
       'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C',
       'misse

## saving version with no prosequence grouped column

In [27]:
df85.to_csv("df85_unique_NOprosequence_MISMAP_scored_3953.csv", index=False)
df92.to_csv("df92_unique_NOprosequence_MISMAP_scored_3953.csv", index=False)
df94.to_csv("df94_unique_NOprosequence_MISMAP_scored_3953.csv", index=False)
df96.to_csv("df96_unique_NOprosequence_MISMAP_scored_3953.csv", index=False)
df97.to_csv("df97_unique_NOprosequence_MISMAP_scored_3953.csv", index=False)